In [ ]:
import cv2
import numpy as np
import pandas as pd
import function
import utlis

def main():
    webCamFeed = True
    cap = cv2.VideoCapture(1)
    cap.set(10,160)
    og_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    og_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Input resolution: {og_w} x {og_h}")
    heightImg = 480
    widthImg  = 640
    h_rate=og_h/heightImg
    w_rate=og_w/widthImg
    utlis.initializeTrackbars()
    count=0
    while True:

        if webCamFeed:success, img = cap.read()
        ogImage=img.copy()
        img = cv2.resize(img, (widthImg, heightImg))
        imgBlank = np.zeros((heightImg,widthImg, 3), np.uint8)
        config=utlis.valTrackbars() 
        imgGray, imgBlur, imgThreshold, imgDial,imgErode=function.preprocessing(img, config)
        imgContours = img.copy()
        imgBigContour = img.copy()
        contours, hierarchy = cv2.findContours(imgErode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(imgContours, contours, -1, (255,183,178), 10)

        biggest, maxArea = utlis.biggestContour(contours)
        if biggest.size != 0:
            biggest=utlis.reorder(biggest)
            cv2.drawContours(imgBigContour, biggest, -1, (255,183,178), 20)
            imgBigContour = utlis.drawRectangle(imgBigContour,biggest,2)
            pts1 = np.float32(biggest)
            pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]])
            matrix = cv2.getPerspectiveTransform(pts1, pts2)
            imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
            imgWarpColored = cv2.resize(imgWarpColored,(widthImg,heightImg))

            Ogbiggest=utlis.Ogcontour(biggest,w_rate,h_rate)

            ogImage=utlis.drawRectangle(ogImage,Ogbiggest,4)
            cv2.drawContours(ogImage, Ogbiggest, -1, (0,0,255), 20)
            imgresult,imgresult_=function.gen_results(ogImage,Ogbiggest)
            imgWarpGray = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)
            imgAdaptiveThre= cv2.adaptiveThreshold(imgWarpGray, 255, 1, 1, 7, 2)
            imgAdaptiveThre = cv2.bitwise_not(imgAdaptiveThre)
            imgAdaptiveThre=cv2.medianBlur(imgAdaptiveThre,3)

            imageArray = ([imgBlur,imgThreshold,imgContours],
                        [imgBigContour,imgWarpColored,imgAdaptiveThre])

        else:

            imageArray = ([imgBlur,imgThreshold,imgContours],
                        [imgBlank, imgBlank, imgBlank])

        # # LABELS FOR DISPLAY
        # lables = [["Blur","Gray","Threshold","Contours"],
        #           ["Biggest Contour","Warp Prespective","Warp Gray","Adaptive Threshold"]]

        stackedImage = utlis.stackImages(imageArray,0.75)
        cv2.imshow("Result",stackedImage)
        cv2.imshow("original", ogImage)
        if cv2.waitKey(1) & 0xFF == ord('b'):
            cv2.imwrite("/Users/mac/Dev/Computer_Vision/Computer-Vision/ouput/vertical/myImage"+str(count)+".jpg",imgresult)
            cv2.imwrite("/Users/mac/Dev/Computer_Vision/Computer-Vision/ouput/horiziontal/myImage"+str(count)+".jpg",imgresult_)
            cv2.rectangle(stackedImage, ((int(stackedImage.shape[1] / 2) - 230), int(stackedImage.shape[0] / 2) + 50),
                        (1100, 350), (255, 255, 255), cv2.FILLED)
            cv2.putText(stackedImage, "Đã lưu", (int(stackedImage.shape[1] / 2) - 200, int(stackedImage.shape[0] / 2)),
                        cv2.FONT_HERSHEY_DUPLEX, 3, (0, 0, 0), 5, cv2.LINE_AA)
            cv2.imshow('Result', stackedImage)
            cv2.waitKey(300)
            count += 1

if __name__ == "__main__":
    main()

In [1]:
import cv2
import numpy as np
import utlis
import function

webCamFeed = True
cap = cv2.VideoCapture(1)
cap.set(10,160)

og_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
og_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"Resolution: {og_w} x {og_h}")

heightImg = 480
widthImg  = 640
h_rate=og_h/heightImg
w_rate=og_w/widthImg

utlis.initializeTrackbars()
count=0

while True:

    if webCamFeed:success, img = cap.read()
    ogImage=img.copy()
    img = cv2.resize(img, (widthImg, heightImg))
    imgBlank = np.zeros((heightImg,widthImg, 3), np.uint8)
    config=utlis.valTrackbars() 
    imgGray, imgBlur, imgThreshold, imgDial,imgErode=function.preprocessing(img, config)
    imgContours = img.copy()
    imgBigContour = img.copy()
    contours, hierarchy = cv2.findContours(imgErode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(imgContours, contours, -1, (255,0,0), 10)


    biggest, maxArea = utlis.biggestContour(contours)
    if biggest.size != 0:
        biggest=utlis.reorder(biggest)
        cv2.drawContours(imgBigContour, biggest, -1, (255,183,178), 20)
        imgBigContour = utlis.drawRectangle(imgBigContour,biggest,2)
        pts1 = np.float32(biggest)
        pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]])
        matrix = cv2.getPerspectiveTransform(pts1, pts2)
        imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
        imgWarpColored = cv2.resize(imgWarpColored,(widthImg,heightImg))

        Ogbiggest=utlis.Ogcontour(biggest,w_rate,h_rate)

        ogImage=utlis.drawRectangle(ogImage,Ogbiggest,8)
        cv2.drawContours(ogImage, Ogbiggest, -1, (0,0,255), 20)
        imgresult,imgresult_=function.gen_results(ogImage,Ogbiggest)

        # resultH=297*3
        # resultW=210*3
        # pts1_=np.float32(Ogbiggest)
        # pts2_ = np.float32([[0, 0],[resultW, 0], [0, resultH],[resultW, resultH]])
        # matrix_ = cv2.getPerspectiveTransform(pts1_, pts2_)
        # imgresult = cv2.warpPerspective(ogImage, matrix_, (resultW,resultH))
        # imgresult= cv2.resize(imgresult,(resultW,resultH))
        # imgresult = cv2.cvtColor(imgresult,cv2.COLOR_BGR2GRAY)
        # imgresult=imgresult[20:imgresult.shape[0] - 20, 20:imgresult.shape[1] - 20]
        # imgresult= cv2.adaptiveThreshold(imgresult, 255, 1, 1, 35, 2)
        # # _,imgresult=cv2.threshold(imgresult, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        # imgresult = cv2.bitwise_not(imgresult)
        # imgresult=cv2.medianBlur(imgresult,3)

        # resultW=297*3
        # resultH=210*3
        # pts1_=np.float32(Ogbiggest)
        # pts2_ = np.float32([[0, 0],[resultW, 0], [0, resultH],[resultW, resultH]])
        # matrix_ = cv2.getPerspectiveTransform(pts1_, pts2_)
        # imgresult_ = cv2.warpPerspective(ogImage, matrix_, (resultW,resultH))
        # imgresult_= cv2.resize(imgresult_,(resultW,resultH))
        # imgresult_ = cv2.cvtColor(imgresult_,cv2.COLOR_BGR2GRAY)
        # imgresult_=imgresult_[20:imgresult_.shape[0] - 20, 20:imgresult_.shape[1] - 20]
        # imgresult_= cv2.adaptiveThreshold(imgresult_, 255, 1, 1, 35, 2)
        # # _,imgresult_=cv2.threshold(imgresult_, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        # imgresult_ = cv2.bitwise_not(imgresult_)
        # imgresult_=cv2.medianBlur(imgresult_,3)

        # APPLY ADAPTIVE THRESHOLD
        imgWarpGray = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)
        imgAdaptiveThre= cv2.adaptiveThreshold(imgWarpGray, 255, 1, 1, 7, 2)
        imgAdaptiveThre = cv2.bitwise_not(imgAdaptiveThre)
        imgAdaptiveThre=cv2.medianBlur(imgAdaptiveThre,3)

        # Image Array for Display
        imageArray = ([imgBlur,imgThreshold,imgContours],
                      [imgBigContour,imgWarpColored,imgAdaptiveThre])

    else:

        imageArray = ([imgBlur,imgThreshold,imgContours],
                      [imgBlank, imgBlank, imgBlank])


    lables = [["Blured&Gray","Threshold","Contours"],
              ["Biggest Contour","Warp Prespective","Adaptive Threshold"]]

    stackedImage = utlis.stackImages(imageArray,0.75,lables)
    cv2.imshow("Result",stackedImage)
    cv2.imshow("original", ogImage)
    if cv2.waitKey(1) & 0xFF == ord('b'):
        cv2.imwrite("/Users/mac/Dev/Computer_Vision/Computer-Vision/ouput/vertical/myImage"+str(count)+".jpg",imgresult)
        cv2.imwrite("/Users/mac/Dev/Computer_Vision/Computer-Vision/ouput/horiziontal/myImage"+str(count)+".jpg",imgresult_)
        cv2.rectangle(stackedImage, ((int(stackedImage.shape[1] / 2) - 230), int(stackedImage.shape[0] / 2) + 50),
                      (1100, 350), (255, 255, 255), cv2.FILLED)
        cv2.putText(stackedImage, "Đã lưu", (int(stackedImage.shape[1] / 2) - 200, int(stackedImage.shape[0] / 2)),
                    cv2.FONT_HERSHEY_DUPLEX, 3, (0, 0, 0), 5, cv2.LINE_AA)
        cv2.imshow('Result', stackedImage)
        cv2.waitKey(300)
        count += 1


2024-05-13 00:03:08.561 Python[6267:297530] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Resolution: 1920 x 1080


2024-05-13 00:03:09.231 Python[6267:297530] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


KeyboardInterrupt: 

In [1]:
import sys
sys.path.append("/Users/mac/Dev/Computer_Vision/Computer-Vision/My-project/function.py")

In [3]:
import cv2
print(cv2.__version__)

4.9.0
